In [15]:
import pandas as pd
import numpy as np
import requests
import json
import os
from matplotlib import pyplot as plt
from scipy.stats import linregress
from config import username, api_key

Load in the city data file we found

In [6]:
city_path = os.path.join("Data","combined_data.csv")
df = pd.read_csv(city_path)
df.head()

,city,state,mean site eui,mean site eui res,mean source eui,mean source eui res,CDD,HDD,Tavg,Trange,Tmax,Tmin,Prcp,Snow
0,Birmingham,AL,143.647642,50.020103,400.595034,96.557022,1933,2770,62.78,22.46,73.94,51.44,5712,8
1,Mobile,AL,107.084979,50.998702,315.907443,117.940363,2537,1656,67.46,18.68,76.64,58.10,6528,2
2,Montgomery,AL,112.048215,NaN,298.702917,NaN,2113,2278,64.58,23.54,76.28,52.88,5280,0
3,Huntsville,AL,116.235841,NaN,362.019120,NaN,1540,3517,59.54,23.90,71.42,47.66,5582,24
4,Tuscaloosa,AL,189.021112,NaN,514.514461,NaN,2348,2338,64.94,22.64,76.28,53.78,5046,2


In [18]:
#Save config information
url = "https://bpd-api.lbl.gov/api/v2/analyze/table"

# Build query URL
# params = {
#     "units":"imperial"
# }

energy_data = pd.DataFrame(df["city"],columns=["city"])
clist = list(energy_data["city"])
electric_res = []
electric_com = []
fuel_res = []
fuel_com = []

headers = {
    "Content-Type":"application/json",
    "Authorization":f"ApiKey {username}:{api_key}",
}

energy_data


,city
0,Birmingham
1,Mobile
2,Montgomery
3,Huntsville
4,Tuscaloosa
5,Anchorage
6,Juneau
7,Phoenix
8,Mesa
9,Tucson


In [20]:
payload = {
        "filters":{"city":["Austin"]},
        "group_by":["building_class"],
        "analyze_by":"electric_eui"
    }
    
response = requests.post(url=url,data=json.dumps(payload),headers=headers,verify=True)
res_json = response.json()
res_json

{'metadata': {'message': 'success',
  'request_time': 0.0656440258026123,
  'url': 'http://bpd.lbl.gov/developers#analyze/table',
  'version': '2.2.1'},
 'summary': {'count': 2743,
  'group': 'ALL',
  'mean': 23.574027456192688,
  'percentile_0': 8.165152374892685e-08,
  'percentile_100': 384.6619999999999,
  'percentile_25': 0.8088142738035257,
  'percentile_50': 16.03455228338081,
  'percentile_75': 28.409448703504466,
  'standard_deviation': 39.012371122899786},
 'table': [{'count': 0,
   'group': [{'field': 'building_class',
     'type': 'categorical',
     'value': 'Unknown'}],
   'mean': None,
   'percentile_0': None,
   'percentile_100': None,
   'percentile_25': None,
   'percentile_50': None,
   'percentile_75': None,
   'standard_deviation': None},
  {'count': 2364,
   'group': [{'field': 'building_class',
     'type': 'categorical',
     'value': 'Residential'}],
   'mean': 13.654886789521315,
   'percentile_0': 8.165152374892685e-08,
   'percentile_100': 206.71153493908554,

In [12]:
res_json["table"][1]["mean"]

19.6446202184613

Calling the API for the data on commercial buildings, site eui, source eui, occupants, occupants density, floor area, operating hours

In [19]:
for city in clist:
    payload = {
        "filters":{"city":[city]},
        "group_by":["building_class"],
        "analyze_by":"fuel_eui"
    }
    
    try:
        response = requests.post(url=url,data=json.dumps(payload),headers=headers,verify=True)
        res_json = response.json()
        fuel_res.append(res_json["table"][1]["mean"])
        fuel_com.append(res_json["table"][2]["mean"])
        print(f"Processing Record : {city}")
        
    except:
        print(f"Missing Data : {city}")
        fuel_res.append(np.nan)
        fuel_com.append(np.nan)
        

Processing Record : Birmingham
Processing Record : Mobile
Processing Record : Montgomery
Processing Record : Huntsville
Processing Record : Tuscaloosa
Processing Record : Anchorage
Missing Data : Juneau
Processing Record : Phoenix
Processing Record : Mesa
Processing Record : Tucson
Processing Record : Yuma
Processing Record : Little Rock
Processing Record : Fayetteville
Processing Record : Fort Smith
Processing Record : Jonesboro
Processing Record : Conway
Processing Record : Los Angeles
Processing Record : San Francisco
Processing Record : San Diego
Processing Record : San Jose
Processing Record : Denver
Processing Record : Colorado Springs
Processing Record : Fort Collins
Processing Record : Pueblo
Processing Record : New Haven
Processing Record : Bridgeport
Processing Record : Hartford
Processing Record : Stamford
Processing Record : Waterbury
Processing Record : Dover
Processing Record : Miami
Processing Record : Tampa
Processing Record : Fort Lauderdale
Processing Record : Jackson

In [21]:
for city in clist:
    payload = {
        "filters":{"city":[city]},
        "group_by":["building_class"],
        "analyze_by":"electric_eui"
    }
    
    try:
        response = requests.post(url=url,data=json.dumps(payload),headers=headers,verify=True)
        res_json = response.json()
        electric_res.append(res_json["table"][1]["mean"])
        electric_com.append(res_json["table"][2]["mean"])
        print(f"Processing Record : {city}")
        
    except:
        print(f"Missing Data : {city}")
        electric_res.append(np.nan)
        electric_com.append(np.nan)
        

Processing Record : Birmingham
Processing Record : Mobile
Processing Record : Montgomery
Processing Record : Huntsville
Processing Record : Tuscaloosa
Processing Record : Anchorage
Processing Record : Juneau
Processing Record : Phoenix
Processing Record : Mesa
Processing Record : Tucson
Processing Record : Yuma
Processing Record : Little Rock
Processing Record : Fayetteville
Processing Record : Fort Smith
Processing Record : Jonesboro
Processing Record : Conway
Processing Record : Los Angeles
Processing Record : San Francisco
Processing Record : San Diego
Processing Record : San Jose
Processing Record : Denver
Processing Record : Colorado Springs
Processing Record : Fort Collins
Processing Record : Pueblo
Processing Record : New Haven
Processing Record : Bridgeport
Processing Record : Hartford
Processing Record : Stamford
Processing Record : Waterbury
Processing Record : Dover
Processing Record : Miami
Processing Record : Tampa
Processing Record : Fort Lauderdale
Processing Record : Ja

In [22]:
energy_data["mean electirc eui"] = electric_com
energy_data["mean electric eui res"] = electric_res
energy_data["mean fuel eui"] = fuel_com
energy_data["mean fuel eui res"] = fuel_res
energy_data

,city,mean electirc eui,mean electric eui res,mean fuel eui,mean fuel eui res
0,Birmingham,143.311563,21.116271,25.618049,NaN
1,Mobile,120.403278,30.169515,10.227344,NaN
2,Montgomery,104.782978,NaN,24.801954,NaN
3,Huntsville,140.163935,27.781260,32.382384,NaN
4,Tuscaloosa,160.009627,NaN,36.074106,NaN
5,Anchorage,90.755170,13.434063,66.884246,NaN
6,Juneau,33.031979,NaN,NaN,NaN
7,Phoenix,76.588709,26.895603,4.228576,5.881798
8,Mesa,123.888037,30.461339,2.630878,0.000000
9,Tucson,119.370537,24.753733,8.777518,9.298245


In [24]:
energy_data.to_csv(os.path.join("Data","bpd_data_additional.csv"),index=False)